
# **Conceptual Graph-Based Recommendation System for PRM Issues: OL Approach**
Welcome to this notebook dedicated to the initial phases of preparing a cutting-edge recommendation system designed to address Public Relations Management (PRM) issues.
Our journey involves a detailed walk-through of the step-by-step process for preparing our PMBOK corpus, extracting descriptions, definitions, keywords, and rules using a combination of cutting-edge techniques such as KeyBERT, GPT-2, SBERT, and BERT.

This data preparation phase is designed to empower PRM professionals with the essential groundwork required to facilitate well-informed decisions and strategy optimization. Join us as we embark on this exciting journey, bridging the gap between data preparation and the ultimate implementation of our recommendation system for PRM.

## Dependencies Installation

In [ ]:
!pip install pdfplumber
!pip install autocorrect
!pip install PyMuPDF
!pip install transformers==4.12.0
!pip install PyPDF2
!pip install transformers
!pip install rouge
!pip install stanza
!pip install keybert
!pip install sentence-transformers
!pip install faiss-cpu --no-cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 81.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 17.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622363 sha256=68e2bc2af9fdb156d2ae80bece787cb63f487c480b4e0dbd2a8e3ee98746dbe7
  Stored in directory: /root/.cache/pip/wheels/b5/7b/6d/b76b29ce11ff8e2521c8c7dd0e5bfee4fb1789d76193124343
Successfully built autocorrect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 36.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 39.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/8

In [ ]:
from rouge import Rouge
import  pdfplumber
import string
import re
import stanza
import pandas as pd
import fitz  # PyMuPDF
import torch
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import CrossEncoder
from transformers import pipeline
from transformers import BertTokenizer, BertForMaskedLM
from transformers import GPT2Tokenizer, GPT2Model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np
import faiss
import time
from pprint import pprint

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Extract Concepts Using Bert

* The Fist step involved data engineering. This process focused on distilling essential information from the raw text to form a well-organized and lucid dataframe, which would serve as the foundation for constructing the knowledge graph and ontology.

* To initiate this phase, we employed the Bert model to identify and extract key concepts ,Description,and Figures. The GPT2 to define the concept ,nd the descriped in

In [ ]:
# Charger le modèle BERT
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
pdf_file_path = '/kaggle/input/prm-pmbok6-2017/PRM-PMBOK6-2017 (1).pdf'

In [ ]:
def extract_titles_from_pdf(pdf_file):
    # Ouvrir le fichier PDF
    pdf_document = fitz.open(pdf_file)
    titles = []
    # Parcourir chaque page du document PDF
    for page_num in range(len(pdf_document)):

        page = pdf_document[page_num]
        page_text = page.get_text()
        # Remplacer les tirets par des sauts de ligne
        page_text=page_text.replace('—','\n')
        # Utilisez une expression régulière pour rechercher des motifs de titre "11.x" ou "11.x.x" ou "11.x.x.x"
        matches = re.findall(r'('+'11+(\.\d+)+)\s+(.+)', page_text)
        # Parcourir les correspondances trouvées

        for match in matches:
            title = match[0]
            description = match[2]
            # Prédire la partie manquante de la description en utilisant BERT
            masked_text = f"[MASK] {description}"
            input_ids = tokenizer.encode(masked_text, add_special_tokens=True, return_tensors="pt")
            mask_index = input_ids[0].tolist().index(tokenizer.mask_token_id)

            with torch.no_grad():
                predictions = model(input_ids)
            predicted_token_id = torch.argmax(predictions.logits[0, mask_index]).item()
            predicted_word = tokenizer.decode(predicted_token_id)

            # Remplacez le masque par le mot prédit dans la description
            description = description.replace("[MASK]", predicted_word)

            # Ajoutez le titre complète à la liste
            titles.append(f"{title} {description}")
    # Fermer le document PDF
    pdf_document.close()
    return titles

# Exemple d'utilisation
titles = extract_titles_from_pdf(pdf_file_path)

# Extract Description

In [ ]:
def extract_description_from_pdf(pdf_file):
        # Ouvrir le fichier PDF
        pdf_document = fitz.open(pdf_file)
        descriptions = {}
        page_text=""
        # Parcourir chaque page du document PDF
        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            page_text += " "+page.get_text()
        page_text=page_text.replace('—',' ')
        page_text=page_text.replace('\n',' ')
        # Fermer le document PDF
        pdf_document.close()
        for concept_indice in range(len(titles)-1) :
                matches = re.findall(titles[concept_indice]+r'\s(.+?)'+titles[concept_indice+1],page_text )

                description = ' '.join(matches)
                # Prédire la partie manquante de la description en utilisant BERT
                masked_text = f"[MASK] {description}"
                input_ids = tokenizer.encode(masked_text, add_special_tokens=True, return_tensors="pt")
                mask_index = input_ids[0].tolist().index(tokenizer.mask_token_id)
                if(input_ids.shape[1]<=512) :
                    with torch.no_grad():
                        predictions = model(input_ids)
                    predicted_token_id = torch.argmax(predictions.logits[0, mask_index]).item()
                    predicted_word = tokenizer.decode(predicted_token_id)

                    # Remplacez le masque par le mot prédit dans la description
                    description = description.replace("[MASK]", predicted_word)

                # Ajoutez descriptions complète à la dictionnaire
                descriptions[titles[concept_indice]]= description
        last=len(titles)-1
        descriptions[titles[last]]=re.findall(titles[last]+r'\s((.+?)+)',page_text )[0][0]

        return descriptions

# Exemple d'utilisation
descriptions = extract_description_from_pdf(pdf_file_path)

Token indices sequence length is longer than the specified maximum sequence length for this model (1789 > 512). Running this sequence through the model will result in indexing errors


# Extract Figure

In [ ]:
def extract_Figure_from_pdf(descriptions):
    # Parcours de chaque clé dans le dictionnaire des descriptions
    figure_with_descriptions = {}
    # Utilisation d'une expression régulière pour rechercher des motifs comme "Figure x-x.x"
    for key in descriptions:
        figures=set()
        # Use a regular expression to search for patterns like "Figure x-x.x"

        matches = re.findall(r'(Figure)+\s([1-9][0-9]?|100)+-+([1-9][0-9]?|100)+.', descriptions[key], re.MULTILINE)

        # Traitement de chaque correspondance trouvé
        for match in matches:
            title = match[0]
            title2 = match[1]
            description = match[2]
            # Prédiction de la partie manquante de la description en utilisant BERT

            masked_text = f"[MASK] {description}"
            input_ids = tokenizer.encode(masked_text, add_special_tokens=True, return_tensors="pt")
            mask_index = input_ids[0].tolist().index(tokenizer.mask_token_id)

            with torch.no_grad():
                predictions = model(input_ids)
            predicted_token_id = torch.argmax(predictions.logits[0, mask_index]).item()
            predicted_word = tokenizer.decode(predicted_token_id)

            # Remplacer le masque par le mot prédit dans la description
            description = description.replace("[MASK]", predicted_word)

            # Ajoutez le figure complète à la liste
            figures.add(f"{title} {title2}-{description}")

        # Assigner l'ensemble de figures à la clé correspondante dans le dictionnaire de résultat
        figure_with_descriptions[key]=figures
    return figure_with_descriptions

# Exemple d'utilisation
Figure = extract_Figure_from_pdf(descriptions)

# Text extraction

In [ ]:
file = open('/kaggle/input/prm-pmbok6-2017/PRM-PMBOK6-2017 (1).pdf','rb')
project_risk_management = ''
with pdfplumber.open(file) as pdf:
    for i in range(0,64):
        page = pdf.pages[i].filter(lambda obj: not (obj["object_type"] == "char" and obj["size"] > 30))
        project_risk_management += page.extract_text()
project_risk_management = project_risk_management.replace('\n','\n ')
project_risk_management = project_risk_management.replace('  ',' ')
project_risk_management = project_risk_management.lower()

# Auto correct

*** using spelling-correction-english-base model we correct the corpus ,we commented it because it is a time consumming process**

In [ ]:
fix_spelling = pipeline("text2text-generation",model="oliverguhr/spelling-correction-english-base")

In [ ]:
""""#Example text (you can replace this with your corpus)
#Process the corpus text
doc = nlp(project_risk_management)
l=[]
#Iterate through each sentence
for sentence in doc.sentences:
    # Access the text of the sentence
    sentence_text = " ".join([word.text for word in sentence.words])

#Print the sentence text
    l.append(fix_spelling(sentence_text,max_length=2048)[0]['generated_text'])
project_risk_management=" ".join(l)    """

'"#Example text (you can replace this with your corpus)\n#Process the corpus text\ndoc = nlp(project_risk_management)\nl=[]\n#Iterate through each sentence\nfor sentence in doc.sentences:\n    # Access the text of the sentence\n    sentence_text = " ".join([word.text for word in sentence.words])\n\n#Print the sentence text\n    l.append(fix_spelling(sentence_text,max_length=2048)[0][\'generated_text\'])\nproject_risk_management=" ".join(l)    '

# Extract Definition using gpt2

In [ ]:
# We load the pretrained GPT-2 model and tokenizer
model_name = "gpt2-medium"  # We choose a GPT-2 variant
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [ ]:
file = open('output.txt','w')
file.write(project_risk_management)
file.close()

In [ ]:
# We load and preprocess the fine-tuning dataset

dataset = TextDataset(
     tokenizer=tokenizer,
     file_path='/kaggle/working/output.txt',
     block_size=128,  # Adjust block size as needed
 )

data_collator = DataCollatorForLanguageModeling(
     tokenizer=tokenizer, mlm=False,
 )

 # We configure training arguments
training_args = TrainingArguments(
     output_dir='./fine-tuned-gpt2',  # Specify the output directory
     overwrite_output_dir=True,
     num_train_epochs=20,  # Adjust the number of training epochs
     per_device_train_batch_size=16,  # Adjust batch size as needed
     save_steps=10_000,  # Specify how often to save the model
 )

 # we initialize the Trainer
trainer = Trainer(
     model=model,
     args=training_args,
     data_collator=data_collator,
     train_dataset=dataset,
 )

# we fine-tune the model
trainer.train()

# we save the fine-tuned model
trainer.save_model()

# we can now use the fine-tuned model for text generation tasks

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


In [ ]:
# we remove numbers from each element while keeping spaces
cleaned_concept_list = [' '.join(filter(str.isalpha, element.split())) for element in titles]

**We executed this code only once because it takes a long time. We used the result of the first execution, and then we commented out this cell to prevent it from slowing down future notebook executions.**

In [ ]:
'''# We load the fine-tuned model
fine_tuned_model = GPT2LMHeadModel.from_pretrained('./fine-tuned-gpt2')  # We load from the fine-tuned model directory

# We set the model to evaluation mode
fine_tuned_model.eval()
definitions={}
for i in range(len(cleaned_concept_list)):

# We define a starting prompt for text generation
    prompt = "define the " + cleaned_concept_list[i] + " concept."

# We tokenize the prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

# We generate text using the fine-tuned model
    output = fine_tuned_model.generate(input_ids, max_length=200, num_return_sequences=1)

# We decode and print the generated text
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    definitions[cleaned_concept_list[i]] = generated_text
#print(definitions)'''

'# We load the fine-tuned model\nfine_tuned_model = GPT2LMHeadModel.from_pretrained(\'./fine-tuned-gpt2\')  # We load from the fine-tuned model directory\n\n# We set the model to evaluation mode\nfine_tuned_model.eval()\ndefinitions={}\nfor i in range(len(cleaned_concept_list)):\n    \n# We define a starting prompt for text generation\n    prompt = "define the " + cleaned_concept_list[i] + " concept."\n\n# We tokenize the prompt\n    input_ids = tokenizer.encode(prompt, return_tensors="pt")\n\n# We generate text using the fine-tuned model\n    output = fine_tuned_model.generate(input_ids, max_length=200, num_return_sequences=1)\n\n# We decode and print the generated text\n    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)\n    definitions[cleaned_concept_list[i]] = generated_text\n#print(definitions)'

# Create concepts dataframe
* **We created our initial dataframe with 6 columns:**
1. **Risk_concepts**: contains the name of the concept.
2. **Relation_Type**: contains the type of relation with another concept or process (e.g. has input, is SubClass of ...).
3. **Descriptions**: contains the text between each concept
4. **Definition**: contains the definition extracted by gpt2
5. **Figure**: contains the isDescribedin part, which refers to a figure.
6. **Described_in**: contains the isDescribedin part, which refers to a section.

In [ ]:
data_frame = pd.DataFrame()
data_frame['risk_concepts'] = [c for c in  titles]
data_frame['Relation_Type'] = ""
data_frame['Descriptions'] = ""
data_frame['Clean_descriptions'] = ""
data_frame['Figure'] = ""
data_frame['Described_in'] = ""
data_frame['Definition']=""

# Extract and add Relation_Type

In [ ]:
for idx, row in data_frame.iterrows():
    if ":" in str(row.risk_concepts) :
            key=str(row.risk_concepts)[str(row.risk_concepts).index(":")+2:].lower
            data_frame.at[idx, 'Relation_Type'] = f'Has {key}'

# Add Figures To data_frame


In [ ]:
datafigures = [{'Concept': titles, 'Figures': ",".join(list(Figure[titles]))}for titles,content in Figure.items()]
concepts_figures_frame = pd.DataFrame(datafigures)

In [ ]:
 for idx, row in data_frame.iterrows():
    concept=str(row.risk_concepts)
    #def_string=""
    #data_frame.at[idx,'risk_concepts'] = concept.lower()
    for i in concepts_figures_frame[concepts_figures_frame["Concept"]==concept]["Figures"].values:
         def_string=str(i)
    data_frame.at[idx,'Figure'] = def_string

# Add the Description To data_frame

In [ ]:
data = [{'Concept': titles, 'Descriptions': descriptions[titles]}for titles,content in descriptions.items()]
concepts_definition_frame = pd.DataFrame(data)


In [ ]:
 for idx, row in data_frame.iterrows():
    concept=str(row.risk_concepts)
    for i in concepts_definition_frame[concepts_definition_frame["Concept"]==concept]["Descriptions"].values:
         def_string=str(i)
    data_frame.at[idx,'Descriptions'] = def_string

# Add the Definition To data_frame

In [ ]:
data_df=pd.read_csv("/kaggle/input/definition-vf/definition_vf.csv")

In [ ]:
for i in range(len(data_df)):
    data_frame['Definition'][i] = data_df['Definition'][i]


In [ ]:
data_frame['Clean_descriptions'] = data_frame['Descriptions']

# Extract the "described in section"

In [ ]:
# Chargez le tokenizer GPT-2 préentraîné
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Chargez le modèle GPT-2 préentraîné
model = GPT2Model.from_pretrained("gpt2")

def extraire_numeros_sections(texte):
    texte=texte.lower()
    # Recherchez tous les textes "described in section" dans le texte
    indices = [match.start() for match in re.finditer("described in section", texte)]

    numeros_sections = set()  # Utilisez un ensemble pour stocker les numéros de section uniques

    for start_idx in indices:
        # Extrait le texte après "described in section"
        texte_apres_section = texte[start_idx + len("described in section"):].strip()

        # Ajoutez une séquence d'arrêt de texte au texte
        texte_apres_section += tokenizer.eos_token

        # Utilisez le tokenizer pour prétraiter le texte
        texte_enc = tokenizer(texte_apres_section, return_tensors="pt")

        # Passez les données au modèle GPT-2 pour l'encodage
        with torch.no_grad():
            outputs = model(**texte_enc)

        # Utilisez une expression régulière pour extraire les numéros de section (x.x.x.x)
        numeros_section_match = re.findall(r'described in section (\d+\.\d+\.\d+\.\d+)', texte)

        if numeros_section_match:
            numeros_sections.update(numeros_section_match)  # Utilisez "update" pour ajouter des éléments à l'ensemble
    return ",".join(["described in section "+ str(i) for i in list(numeros_sections)])
# Appliquez la fonction pour créer une nouvelle colonne contenant les numéros de section extraits
data_frame["Described_in"] = (data_frame["Descriptions"]).apply(extraire_numeros_sections)



In [ ]:
item_list = []
for item in data_frame["Descriptions"]:
    item_list.append(item.split())
item_list_not_splitted = []
for item in data_frame["Descriptions"]:
    item_list_not_splitted.append(item)

# Remove "described in section" and "figure" from clean Description

In [ ]:
for idx,row in data_frame[""!= data_frame["Described_in"]].iterrows():

    data_frame.at[idx,'Clean_descriptions']=row["Clean_descriptions"].lower().replace(row["Described_in"]+".","")

In [ ]:
for idx,row in data_frame[""!= data_frame["Figure"]].iterrows():

    data_frame.at[idx,'Clean_descriptions']=row["Clean_descriptions"].replace(row["Figure"]+".","")

# Pre-processing phase

In [ ]:
processing = pd.DataFrame()
processing['risk_concepts'] = data_frame['risk_concepts']
processing['Relation_Type'] = data_frame['Relation_Type']
processing['Descriptions'] = data_frame['Clean_descriptions']
processing['Clean_descriptions'] = ""
processing['Figure'] = data_frame['Figure']
processing['Described_in'] = data_frame['Described_in']
processing['Definition'] = data_frame['Definition']

In [ ]:
tmp =[]
for item in data_frame['Clean_descriptions']:
    item = item
    tmp.append(item)

***Removing stopwords, digits and punctuation excluding points in order to maintain the semantic integrity***

In [ ]:
pattern = string.punctuation.replace('.','')

In [ ]:
res = ''
for i in range(len(tmp)):
    ###### PUNCTUATION #########################
    tmp[i] = tmp[i].replace(' u ','')
    tmp[i] = tmp[i].replace('uu','')
    tmp[i] = tmp[i].replace('\n','')
    tmp[i] = tmp[i].replace('—','')
    tmp[i] = tmp[i].replace(',',' ')
    tmp[i] = tmp[i].translate(str.maketrans('', '', pattern))
    ###### Digits #############################
    for ele in tmp[i]:
        if ele.isdigit():
            tmp[i] = tmp[i].replace(ele, ' ')

In [ ]:
for i in range(len(tmp)):
    processing['Clean_descriptions'].iloc[i] = tmp[i]

* **Removing special characters**

In [ ]:
def remove_special_characters(text):
    """
        Remove special special characters, including symbols, emojis, and other graphic characters
    """
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
processing['Clean_descriptions'] = processing['Clean_descriptions'].apply(lambda x: remove_special_characters(x))
processing['Clean_descriptions'] = processing['Clean_descriptions'].apply(lambda s : s.lower())

# Lemma using Stanza

In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

In [ ]:
Lemma_df= pd.DataFrame()
for idx,row in processing.iterrows():
    new_row=""
    for sent in nlp(row['Clean_descriptions']).sentences:
            for word in sent.words:
                #print(word)
                #if not word.is_stop:
                new_row+=" "+word.lemma
    Lemma_df.at[idx,'description']=new_row

# Load the REBEL model

REBEL is a text2text model trained by BabelScape by fine-tuning BART for translating a raw input sentence containing entities and implicit relations into a set of triplets that explicitly refer to those relations. It has been trained on more than 200 different relation types.

**This time we changed the input of the rebel to the preprocessed decriptions column in order to get better results as we are asked in the coaching session**

In [ ]:
Triplets = pd.DataFrame()
Triplets['risk_concepts'] = data_frame['risk_concepts']
Triplets['head'] = ""
Triplets['type'] = ""
Triplets['tail'] = ""

In [ ]:
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Babelscape/rebel-large")
model = AutoModelForSeq2SeqLM.from_pretrained("Babelscape/rebel-large")

# From text to KB

The next step is to write a function that is able to parse the strings generated by REBEL and transform them into relation triplets. This function must take into account additional new tokens used while training the model.

In [ ]:
def extract_relations_from_model_output(text):
    # Initialize an empty list to store relations
    relations = []

    # Initialize variables for relation, subject, and object
    relation, subject, relation, object_ = '', '', '', ''

    # Remove unnecessary tokens from the input text
    text = text.strip()
    text_replaced = text.replace("<s>", "").replace("<pad>", "").replace("</s>", "")

    # Initialize a variable to keep track of the current entity (subject, relation, or object)
    current = 'x'

    # Iterate through the tokens in the modified text
    for token in text_replaced.split():
        if token == "<triplet>":
            # If a triplet tag is found, set the current entity to 't' (relation)
            current = 't'

             # If there was a previous relation, add it to the relations list
            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
                relation = '' # Reset relation
            subject = '' # Reset subject
        elif token == "<subj>":
            # If a subject tag is found, set the current entity to 's' (subject)
            current = 's'

            # If there was a previous relation, add it to the relations list

            if relation != '':
                relations.append({
                    'head': subject.strip(),
                    'type': relation.strip(),
                    'tail': object_.strip()
                })
            object_ = '' # Reset object
        elif token == "<obj>":
            # If an object tag is found, set the current entity to 'o' (object)
            current = 'o'
            relation = '' # Reset relation
        else:
            # Process tokens based on the current entity type
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    # After the loop, check if there are any remaining entities
    if subject != '' and relation != '' and object_ != '':
        relations.append({
            'head': subject.strip(),
            'type': relation.strip(),
            'tail': object_.strip()
        })
    # Return the list of relations
    return relations

In [ ]:
class KB():
    def __init__(self):
        # Initialize a Knowledge Base with an empty list of relations
        self.relations = []

    def are_relations_equal(self, r1, r2):
        # Check if two relations r1 and r2 are equal by comparing their attributes
        return all(r1[attr] == r2[attr] for attr in ["head", "type", "tail"])

    def exists_relation(self, r1):
       # Check if a given relation r1 already exists in the Knowledge Base
        return any(self.are_relations_equal(r1, r2) for r2 in self.relations)

    def add_relation(self, r):
        # Add a relation r to the Knowledge Base if it doesn't already exist
        if not self.exists_relation(r):
            self.relations.append(r)

    def save(self):
        # Save the relations in the Knowledge Base as a list of dictionaries
        dict_list = []
        for r in self.relations:
            dict_list.append(r)
        return(dict_list)

In [ ]:
def from_small_text_to_kb(text, verbose=False):
    kb = KB()

    # Tokenizer text
    model_inputs = tokenizer(text, max_length=512, padding=True, truncation=True,
                            return_tensors='pt')
    if verbose:
        print(f"Num tokens: {len(model_inputs['input_ids'][0])}")

    # Generate
    gen_kwargs = {
        "max_length": 1000,
        "length_penalty": 0,
        "num_beams": 3,
        "num_return_sequences": 3
    }

    generated_tokens = model.generate(
        **model_inputs,
        **gen_kwargs,
    )

    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)

    # create kb
    for sentence_pred in decoded_preds:
        relations = extract_relations_from_model_output(sentence_pred)
        for r in relations:
            kb.add_relation(r)

    return kb

In [ ]:
from tqdm import tqdm

In [ ]:
relation_list =[]
for item in tqdm(Lemma_df['description']):
    kb = from_small_text_to_kb(item, verbose=False) #verbose = True to return the number of tokens in each sentence
    relation_list.append(kb.save())

100%|██████████| 117/117 [02:59<00:00,  1.54s/it]


In [ ]:
dict_list =[]
for item in tqdm(relation_list):
    for dic in item:
        dict_list.append(dic)
triplet = pd.DataFrame(dict_list, columns=['head', 'type', 'tail'])

100%|██████████| 117/117 [00:00<00:00, 186590.71it/s]


In [ ]:
triplet

,head,type,tail
0,risk management,studies,project
1,project,studied by,risk management
2,project,facet of,risk management
3,project,use,risk management
4,project risk,facet of,project risk
...,...,...,...
358,risk audit,facet of,risk management
359,risk audit,part of,risk management process
360,risk register,part of,risk management plan
361,risk register,facet of,risk management plan


In [ ]:
unique_triplets = triplet.drop_duplicates()

In [ ]:
unique_triplets

,head,type,tail
0,risk management,studies,project
1,project,studied by,risk management
2,project,facet of,risk management
3,project,use,risk management
4,project risk,facet of,project risk
...,...,...,...
356,change request,use,change control
357,risk audit,part of,risk management
358,risk audit,facet of,risk management
359,risk audit,part of,risk management process


In [ ]:
tiplet_unique = triplet['head'].unique()

In [ ]:
final_triplet_unique = triplet.drop_duplicates(keep='first')

In [ ]:
final_triplet_unique = final_triplet_unique.reset_index(drop=True)

In [ ]:
final_triplet_unique

,head,type,tail
0,risk management,studies,project
1,project,studied by,risk management
2,project,facet of,risk management
3,project,use,risk management
4,project risk,facet of,project risk
...,...,...,...
249,change request,use,change control
250,risk audit,part of,risk management
251,risk audit,facet of,risk management
252,risk audit,part of,risk management process


# Final data frame

In [ ]:
df_final = pd.DataFrame()
df_final['Concepts'] = [c.lower() for c in data_frame['risk_concepts']]
df_final['Type_relation'] = ''
df_final['Concept_of_type_relation'] = ''
df_final['Descriptions'] = processing['Clean_descriptions']
df_final['Synonym'] = ''
df_final['Reference'] = processing['Figure'] + processing['Described_in']
df_final['Definition'] = processing['Definition']

# Adding final_triplet_unique relation and its range


*This function counts the most frequently occurring head from a list within a given concept.
It returns the head with the maximum length.*

In [ ]:
def count_most_freq(concept , list_head):
    # Initialize an empty dictionary to store counts of each head
    d={}

    # Iterate through each head in the list of heads
    for head in list_head:
        count=0  #Initialize count for the current head

        # Check if the head is present in the concatenation string
        if head in concept :
            count+=1 # Increment count if head is found
            d[head]=count # Store the count in the dictionary

    # Initialize a variable to keep track of the head with the maximum length
    max_lenght=""

    # Iterate through the keys in the dictionary
    for key in d :

        # Compare the length of the current key with the maximum length
        if len(key) >len(max_lenght):
            max_lenght=key # Update the maximum length if needed

    return max_lenght # Return the head with the maximum length

In [ ]:
for idx2,row2 in df_final.iterrows():
    # Loop through each row in the DataFrame 'df_final'

    # Extract the type of relation associated with the most frequent head
    type_relation=list(final_triplet_unique[final_triplet_unique["head"]==count_most_freq(row2['Concepts'],final_triplet_unique['head'].values)]["type"].values)
    if len(type_relation)!=0:
        df_final.at[idx2,'Type_relation']=max(set(type_relation), key = type_relation.count)
        df_final.at[idx2,'Concept_of_type_relation']=final_triplet_unique[final_triplet_unique["head"]==count_most_freq(row2['Concepts'],final_triplet_unique['head'].values)][final_triplet_unique["type"]==df_final.at[idx2,'Type_relation']]["tail"].values

/tmp/ipykernel_32/3879485198.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_final.at[idx2,'Concept_of_type_relation']=final_triplet_unique[final_triplet_unique["head"]==count_most_freq(row2['Concepts'],final_triplet_unique['head'].values)][final_triplet_unique["type"]==df_final.at[idx2,'Type_relation']]["tail"].values
/tmp/ipykernel_32/3879485198.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_final.at[idx2,'Concept_of_type_relation']=final_triplet_unique[final_triplet_unique["head"]==count_most_freq(row2['Concepts'],final_triplet_unique['head'].values)][final_triplet_unique["type"]==df_final.at[idx2,'Type_relation']]["tail"].values
/tmp/ipykernel_32/3879485198.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_final.at[idx2,'Concept_of_type_relation']=final_triplet_unique[final_triplet_unique["head"]==count_most_freq(row2['Concepts'],final_triplet_unique['head'].values)][

# Similarity case

**We updated our approach for similarity by using SBERT to calculate similarities between each concept and the entire description column. Our threshold for considering a similarity is set at 0.75.**

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# Example DataFrame with a "description" column
for idx ,row in df_final.iterrows() :
    # Define the target concept
    target_concept=" ".join(row["Concepts"].split(" ")[1:])

    # Tokenize the target concept
    target_tokens = target_concept.split()

    # Encode the target concept
    target_concept_embedding = model.encode(target_concept, convert_to_tensor=True)

    # Find synonyms for the target concept

    synonyms = []
    for description in Lemma_df["description"]:
        # Tokenize the description
        description_tokens = description.split()

        # Calculate cosine similarity between the target concept and the description
        similarity_scores = util.pytorch_cos_sim(target_concept_embedding, model.encode(description, convert_to_tensor=True))

        # Choose a similarity threshold based on your needs
        threshold = 0.75

        # Check if the similarity score is above the threshold
        if similarity_scores > threshold and description != target_concept:
            synonyms.append(description)

    df_final.at[idx,"Synonym"]=synonyms

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

# Key Bert

**When using KeyBERT, we first extracted keywords from each description, following a process of lemmatization and stop word removal. To enhance the quality of our results, we fine-tuned specific model parameters such as the diversity threshold and n-gram range. This optimization allowed us to achieve the best possible results.**

In [ ]:
df_final['Keywords']=" "

In [ ]:
kw_model = KeyBERT()
#df_final = df_final.reset_index(drop=False)
for i in range(len(df_final)):
    row_kw=[]
    kwords=kw_model.extract_keywords(Lemma_df['description'][i],keyphrase_ngram_range=(2,3), top_n=4,stop_words="english",threshold=.85,diversity=0.5)
    for j in range(len(kwords)):
        row_kw.append(kwords[j][0])
    df_final['Keywords'].iloc[i] = row_kw

# Evaluation


*The Rouge metric, used here, assesses the quality of text summarization and other language generation tasks. It calculates how well the generated text matches the reference text in our case the PMBOK , providing insights into the system's effectiveness. The output includes precision, recall, and the F-measure, which collectively gauge the quality of the generated content.*

In [ ]:
all_data_combined = df_final['Concepts'].tolist() + df_final['Type_relation'].tolist()+ df_final['Definition'].tolist()+df_final['Synonym'].tolist()+df_final['Reference'].tolist()+df_final['Descriptions'].tolist()

In [ ]:
ch=''
for e in all_data_combined:
    ch += '. '+str(e)

In [ ]:
proc = project_risk_management

In [ ]:
rouge = Rouge()
score=rouge.get_scores(ch, proc, avg=True)
print('Precision = '+str(score['rouge-1']['p']))
print('Recall = ' +str(score['rouge-1']['r']))
print('f-measure = '+str(score['rouge-1']['f']))

Precision = 0.8233102714209686
Recall = 0.724250936329588
f-measure = 0.7706102067265885


In [ ]:
df_final

,Concepts,Type_relation,Concept_of_type_relation,Descriptions,Synonym,Reference,Definition,Keywords
0,11.1 plan risk management,subclass of,risk management,the process of defining how to conduct risk ma...,[ plan risk management be the process of dene ...,,the risk management concept defines the areas ...,"[risk management activity, risk management, co..."
1,11.2 identify risks,has part,identify risk process,the process of identifying individual project ...,[],,it can be used to identify individual project ...,"[project risk document, individual project ris..."
2,11.3 perform qualitative risk analysis,subclass of,"[quantitative risk analysis, organizational pr...",the process of prioritizing individual project...,[ perform qualitative risk analysis be the pro...,,the perform qualitative risk analysis concept ...,"[project risk analysis, individual project ris..."
3,11.4 perform quantitative risk analysis,subclass of,"[qualitative risk analysis, organizational pro...",the process of numerically analyzing the combi...,[],,the perform quantitative risk analysis concept...,"[individual project risk, uncertainty overall ..."
4,11.5 plan risk responses,facet of,project management plan,the process of developing options selecting s...,[ the process of implement agreedupon risk res...,,the risk responses concept defines the general...,"[develop option select, develop option, indivi..."
...,...,...,...,...,...,...,...,...
112,11.7.3.1 work performance information,,,work performance information includes informa...,[],described in section 4.5.1.3,the work performance information concept defin...,"[work performance information, project risk ma..."
113,11.7.3.2 change requests,facet of,"[change control, change control process]",the monitor risks process may result in a cha...,[],described in section 4.3.3.4,described in section 4.4.3.1. the change reque...,"[monitor risk process, risk process, change re..."
114,11.7.3.3 project management plan updates,has part,"[project risk management, requirement manageme...",any change to the project management plan goes...,[],,i. project management plan updates describe th...,"[change project management, project management..."
115,11.7.3.4 project documents updates,facet of,project,project documents that may be updated as a res...,[],"described in section 11.2.3.1,described in sec...","i files describe the current project status, o...","[project risk register, risk report section, u..."
